## Scratch

In [84]:
import numpy as np
import torch

class Regression1:
    def __init__(self, input, target):
        self.input = input
        self.target = target
        self.pred = torch.randn(target.size()[0], target.size()[1])
        self.init_wb()
        self.loss = float('inf')

    def init_wb(self):
        self.w = torch.randn(self.target.size()[1], self.input.size()[1], requires_grad=True)
        self.b = torch.randn(self.target.size()[1], requires_grad=True)
    
    def model(self):
        return self.input @ self.w.t() + self.b
    
    def mse(self):
        diff = self.pred - self.target
        return torch.sum(diff * diff) / diff.numel()
    
    def train(self, epoch):
        for i in range(epoch):
            self.pred = self.model()
            self.loss = self.mse()
            self.loss.backward()
            with torch.no_grad():
                self.w -= self.w.grad * 1e-5
                self.b -= self.b.grad * 1e-5
                self.w.grad.zero_()
                self.b.grad.zero_()
                
            if (i+1) % 10 == 0:
                print('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, epoch, self.loss))

In [111]:
input = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

target = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

input = torch.from_numpy(input)
target = torch.from_numpy(target)

In [86]:
r1 = Regression1(input, target)

print(r1.input.shape)
print(r1.target.shape)

torch.Size([5, 3])
torch.Size([5, 2])


In [87]:
print(r1.loss)
print(r1.target)
print(r1.pred)

inf
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
tensor([[ 0.4723, -1.3326],
        [ 0.6007, -0.0127],
        [-0.8366,  0.3843],
        [-0.7716,  0.8846],
        [-0.0068, -1.0080]])


In [88]:
r1.train(200)

Epoch [10/200], Loss: 1281.0388
Epoch [20/200], Loss: 277.4514
Epoch [30/200], Loss: 239.7493
Epoch [40/200], Loss: 222.6698
Epoch [50/200], Loss: 207.7260
Epoch [60/200], Loss: 194.3576
Epoch [70/200], Loss: 182.3743
Epoch [80/200], Loss: 171.6148
Epoch [90/200], Loss: 161.9367
Epoch [100/200], Loss: 153.2149
Epoch [110/200], Loss: 145.3386
Epoch [120/200], Loss: 138.2108
Epoch [130/200], Loss: 131.7457
Epoch [140/200], Loss: 125.8677
Epoch [150/200], Loss: 120.5101
Epoch [160/200], Loss: 115.6140
Epoch [170/200], Loss: 111.1280
Epoch [180/200], Loss: 107.0060
Epoch [190/200], Loss: 103.2079
Epoch [200/200], Loss: 99.6981


In [89]:
print(r1.loss)
print(r1.pred)

tensor(99.6981, grad_fn=<DivBackward0>)
tensor([[ 59.1608,  72.0354],
        [ 75.7687,  96.1213],
        [130.1261, 140.5266],
        [ 32.2207,  47.0876],
        [ 84.1538, 105.2219]], grad_fn=<AddBackward0>)


## Pytorch

In [33]:
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [105]:
class Regression2:
    def __init__(self, input, target):
        self.batch_size = 5
        self.lr = 1e-5
        self.loss = float('inf')
        self.init_data(input, target)
        self.init_model(input.size()[1], target.size()[1])
        
    def init_data(self, input, target):
        train_ds = TensorDataset(input, target)
        self.train_dl = DataLoader(train_ds, self.batch_size, shuffle=True)

    def init_model(self, x_col, y_col):
        self.model = nn.Linear(x_col, y_col)
    
    def cal_loss(self, pred, target):
        loss_fn = F.mse_loss
        return loss_fn(pred, target)
    
    def init_opt(self):
        self.opt = torch.optim.SGD(self.model.parameters(), lr=self.lr)
    
    def train(self, total_epoch):
        self.init_opt()
        for epoch in range(total_epoch):
            for input, target in self.train_dl:
                pred = self.model(input)
                self.loss = self.cal_loss(pred, target)
                self.loss.backward()
                self.opt.step()
                self.opt.zero_grad()

            if (epoch+1) % 10 == 0:
                print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, total_epoch, self.loss.item()))

In [110]:
input = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
target = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

input = torch.from_numpy(input)
target = torch.from_numpy(target)

In [107]:
r2 = Regression2(input, target)

In [108]:
r2.train(200)

Epoch [10/200], Loss: 526.7911
Epoch [20/200], Loss: 641.8920
Epoch [30/200], Loss: 181.6533
Epoch [40/200], Loss: 187.2015
Epoch [50/200], Loss: 200.2147
Epoch [60/200], Loss: 35.0323
Epoch [70/200], Loss: 75.4663
Epoch [80/200], Loss: 32.3763
Epoch [90/200], Loss: 27.0763
Epoch [100/200], Loss: 21.8047
Epoch [110/200], Loss: 25.4438
Epoch [120/200], Loss: 17.2758
Epoch [130/200], Loss: 16.2277
Epoch [140/200], Loss: 9.5433
Epoch [150/200], Loss: 9.7288
Epoch [160/200], Loss: 9.6786
Epoch [170/200], Loss: 8.0269
Epoch [180/200], Loss: 4.5977
Epoch [190/200], Loss: 3.8275
Epoch [200/200], Loss: 5.7100


In [109]:
preds = r2.model(input)
preds

tensor([[ 56.9838,  70.8880],
        [ 81.7998,  98.8292],
        [118.8965, 135.8326],
        [ 21.2904,  40.1613],
        [101.0646, 114.1874],
        [ 55.7403,  69.8107],
        [ 81.5953,  98.5983],
        [119.1650, 136.2637],
        [ 22.5339,  41.2386],
        [102.1037, 115.0339],
        [ 56.7793,  70.6572],
        [ 80.5563,  97.7518],
        [119.1010, 136.0634],
        [ 20.2513,  39.3148],
        [102.3081, 115.2648]], grad_fn=<AddmmBackward>)

In [104]:
r2.model(torch.tensor([[75, 63, 44.]]))

tensor([[53.7683, 67.4697]], grad_fn=<AddmmBackward>)